# Check Loglikelihood model selection behaviour

We need to verify if the LL is correlated with the Mixture model summary stats fits. If they are not, something went wrong somewhere, or the parameters are interacting in a weird way that we need to understand...

Performed by *generator_LLcheck_random_fitexperimentallt_sigmaxMratiosigmaoutput_121015.py*


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from experimentlauncher import ExperimentLauncher
from dataio import DataIO
import plots_experimental_data

# import matplotlib.animation as plt_anim
from mpl_toolkits.mplot3d import Axes3D

import re
import inspect
import imp

import utils
# import submitpbs
import load_experimental_data

In [4]:
%run reloader_LLcheck_random_fitexperimentallt_sigmaxMratiosigmaoutput_121015.py

ERROR: File `u'reloader_LLcheck_random_fitexperimentallt_sigmaxMratiosigmaoutput_121015.py'` not found.


In [3]:
#### SETUP
#
plots_bestfits = True
plots_scatter3d = False

savefigs = True

# dataio = DataIO(output_folder=generator_module.pbs_submission_infos['simul_out_dir'] + '/outputs/', label='global_' + dataset_infos['save_output_filename'])

#
#### /SETUP